In [13]:
import pandas as pd

# Caminhos dos arquivos
input_csv = 'TAGS_noCategorizeds.csv'  # Nome do CSV com os dados de entrada
categories_txt = 'Categorias para TAGs.txt'  # Nome do arquivo TXT com as categorias
output_csv = 'categorized_artifacts.csv'  # Nome do arquivo de saída

# Função para carregar categorias do arquivo TXT
def load_categories(txt_file):
    with open(txt_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    category_mapping = {}
    current_category = None
    
    for line in lines:
        line = line.strip()
        if line.endswith(':'):
            current_category = line[:-1]  # Define a categoria atual
        elif line.startswith('#') and current_category:
            tag = line.strip()
            category_mapping[tag] = current_category  # Mapeia a tag para a categoria atual
    
    return category_mapping

# Carrega o mapeamento de categorias
category_mapping = load_categories(categories_txt)

# Carrega o CSV de entrada
data = pd.read_csv(input_csv)

# Função para mapear múltiplas tags para categorias
def map_tags_to_categories(tags):
    categories = []
    for tag in tags:
        if tag in category_mapping:
            categories.append(category_mapping[tag])
    return categories

# Converte a coluna TAG (que pode ter múltiplas tags) em categorias
data['CategoryList'] = data['TAG'].apply(lambda x: map_tags_to_categories(eval(x)))

# Explode a lista de categorias para facilitar o agrupamento
data = data.explode('CategoryList')

# Renomeia a coluna para algo mais claro
data = data.rename(columns={'CategoryList': 'Category'})

# Agrupa os dados por artefato não categorizado e determina a categoria predominante
grouped = data.groupby('ArtifactID')

# Cria uma lista para armazenar os resultados
results = []

for artifact, group in grouped:
    if not group['Category'].empty:
        mode_value = group['Category'].mode()
        if not mode_value.empty:
            results.append({'ArtifactID': artifact, 'Categoria': mode_value[0]})

# Converte os resultados para um DataFrame
result_df = pd.DataFrame(results)

# Salva o resultado em um novo arquivo CSV
result_df.to_csv(output_csv, index=False)
print(f"Arquivo gerado com sucesso: {output_csv}")


Arquivo gerado com sucesso: categorized_artifacts.csv
